In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # import tqdm for progress bar 
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
target_labels = ['Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___healthy', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)']

In [3]:
main_path = 'data_penyakit/dataset/train'

In [48]:
# Load the images and labels
filename = []
X = []
y = []
for label in target_labels:
    label_dir = os.path.join(main_path, label)
    for img_file in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_file)
        img = load_img(img_path, target_size=(128, 128))
        img_array = img_to_array(img)
        filename.append(img_file)
        X.append(img_array)
        y.append(target_labels.index(label))

In [5]:
from tensorflow.keras.utils import to_categorical

In [6]:
X = np.array(X)
y = to_categorical(y, num_classes=len(target_labels))

In [7]:
X.shape

(7222, 128, 128, 3)

In [8]:
y.shape

(7222, 4)

In [51]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test, filename_train, filename_test = train_test_split(X, y, filename, test_size=0.2, random_state=42)

In [10]:
X_train.shape

(5777, 128, 128, 3)

In [11]:
X_test.shape

(1445, 128, 128, 3)

In [12]:
y_train.shape

(5777, 4)

In [13]:
y_test.shape

(1445, 4)

In [14]:
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.models import Sequential

In [15]:
len(target_labels)

4

In [16]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size= (3,3), activation = 'relu',input_shape=(128,128,3)))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(96,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(32,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())

cnn_model.add(Dropout(0.2))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation = 'relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(len(target_labels), activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# Train the CNN model
cnn_history = cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10


2023-05-31 09:45:51.998624: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


181/181 [==============================] - 29s 158ms/step - loss: 0.4160 - accuracy: 0.8430 - val_loss: 1.9232 - val_accuracy: 0.4817
Epoch 2/10
181/181 [==============================] - 34s 186ms/step - loss: 0.1566 - accuracy: 0.9441 - val_loss: 0.0669 - val_accuracy: 0.9737
Epoch 3/10
181/181 [==============================] - 32s 177ms/step - loss: 0.0977 - accuracy: 0.9675 - val_loss: 0.5389 - val_accuracy: 0.8913
Epoch 4/10
181/181 [==============================] - 33s 183ms/step - loss: 0.0613 - accuracy: 0.9766 - val_loss: 0.3813 - val_accuracy: 0.9052
Epoch 5/10
181/181 [==============================] - 32s 178ms/step - loss: 0.0466 - accuracy: 0.9818 - val_loss: 2.1043 - val_accuracy: 0.5654
Epoch 6/10
181/181 [==============================] - 33s 182ms/step - loss: 0.0436 - accuracy: 0.9856 - val_loss: 0.6629 - val_accuracy: 0.8263
Epoch 7/10
181/181 [==============================] - 34s 187ms/step - loss: 0.0519 - accuracy: 0.9825 - val_loss: 1.2125 - val_accuracy: 0.7

In [18]:
# Evaluate the performance of the trained CNN model on the test set
cnn_scores = cnn_model.evaluate(X_test, y_test, verbose=0)
print("CNN Model Accuracy: %.2f%%" % (cnn_scores[1] * 100))

CNN Model Accuracy: 98.69%


In [19]:
cnn_scores

[0.03419118747115135, 0.9868512153625488]

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Use the model to predict the test dataset
y_pred_prob = cnn_model.predict(X_test)

# Convert probabilities to classes
y_pred = np.argmax(y_pred_prob, axis=1)

# If y_test is one-hot encoded, convert it to classes as well
y_test_classes = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_classes, y_pred)

# Calculate precision
precision = precision_score(y_test_classes, y_pred, average='weighted') 

# Calculate recall
recall = recall_score(y_test_classes, y_pred, average='weighted') 

# Calculate F1 score
f1 = f1_score(y_test_classes, y_pred, average='weighted')

# Print the metrics
print("Accuracy: %.2f%%" % (accuracy * 100))
print("Precision: %.2f%%" % (precision * 100))
print("Recall: %.2f%%" % (recall * 100))
print("F1 score: %.2f%%" % (f1 * 100))

46/46 [==============================] - 2s 52ms/step
Accuracy: 98.69%
Precision: 98.73%
Recall: 98.69%
F1 score: 98.69%


In [52]:
from tabulate import tabulate

# Create a DataFrame with true labels, predicted labels, and target labels
df = pd.DataFrame({'Filename': filename_test,
                   'True Labels': [target_labels[label] for label in y_test_classes],
                   'Predicted Labels': [target_labels[label] for label in y_pred]})

# Convert DataFrame to tabular format
table = tabulate(df, headers='keys', tablefmt='psql')

# Print the table
print(table)

# Save the DataFrame to a CSV file
df.to_csv('labels_predictions-cnn-anggur.csv', index=False)

+------+--------------------------------------------------------------------------+--------------------------------------------+--------------------------------------------+
|      | Filename                                                                 | True Labels                                | Predicted Labels                           |
|------+--------------------------------------------------------------------------+--------------------------------------------+--------------------------------------------|
|    0 | 4c08f1e5-825d-4992-8cb7-0ac1bdf38ae2___FAM_L.Blight 1604_flipLR.JPG      | Grape___Leaf_blight_(Isariopsis_Leaf_Spot) | Grape___Leaf_blight_(Isariopsis_Leaf_Spot) |
|    1 | e671f7da-5f56-4409-8239-e31500250709___FAM_B.Rot 0743_flipLR.JPG         | Grape___Black_rot                          | Grape___Black_rot                          |
|    2 | b6c619c6-fd1e-41f3-80c8-8b35c2fa87ee___Mt.N.V_HL 8973.JPG                | Grape___healthy                            | G

print only for false predict

In [53]:
# Filter rows where true label and predicted label do not match
false_predictions = df[df['True Labels'] != df['Predicted Labels']]

# Convert DataFrame to tabular format
table = tabulate(false_predictions, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table)

# Save the false predictions to a CSV file
false_predictions.to_csv('false_predictions-cnn-anggur.csv', index=False)

False Predictions:
+------+---------------------------------------------------------------------+--------------------------------------------+--------------------------------------------+
|      | Filename                                                            | True Labels                                | Predicted Labels                           |
|------+---------------------------------------------------------------------+--------------------------------------------+--------------------------------------------|
|   20 | 069123c0-8f04-499e-a2ac-687e24b099f4___FAM_B.Msls 3917_flipLR.JPG   | Grape___Esca_(Black_Measles)               | Grape___Black_rot                          |
|   49 | e9b5a691-e85f-48fc-8135-94b4147e9bda___FAM_B.Msls 4231.JPG          | Grape___Esca_(Black_Measles)               | Grape___Black_rot                          |
|  124 | b6303273-6a10-4bb0-a4f4-7ee0cf2c782d___FAM_B.Msls 1789.JPG          | Grape___Esca_(Black_Measles)               | Grape___Blac

In [21]:
# Extract the features using the trained CNN model
cnn_features = cnn_model.predict(X_train)

181/181 [==============================] - 10s 56ms/step


# RF

In [22]:
import time

In [23]:
# import the Random Forest classifier
from RF_Manual import RandomForest

In [24]:
# Train a Random Forest classifier on the extracted features
start_time = time.time()
rf_classifier = RandomForest(n_trees=100, max_depth=42)
rf_classifier.fit(cnn_features, np.argmax(y_train, axis=1))

# count time for training
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

Training time: 20.70679807662964 seconds


In [25]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Evaluate the performance of the Random Forest classifier on the test set
rf_features = cnn_model.predict(X_test)
rf_features = rf_features.reshape(rf_features.shape[0], -1)
rf_predictions = rf_classifier.predict(rf_features)
rf_accuracy = np.mean(rf_predictions == np.argmax(y_test, axis=1))
print("Random Forest Classifier Accuracy: %.2f%%" % (rf_accuracy * 100))

# Calculate and print the Precision
rf_precision = precision_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Precision: %.2f" % rf_precision)

# Calculate and print the Recall
rf_recall = recall_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Recall: %.2f" % rf_recall)

# Calculate and print the F1 score
rf_f1_score = f1_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier F1 Score: %.2f" % rf_f1_score)

46/46 [==============================] - 2s 45ms/step
Random Forest Classifier Accuracy: 99.58%
Random Forest Classifier Precision: 1.00
Random Forest Classifier Recall: 1.00
Random Forest Classifier F1 Score: 1.00


In [54]:
# Create a DataFrame with true labels and predicted labels
df_rf = pd.DataFrame({'Filename': filename_test,
                      'True Labels': [target_labels[label] for label in y_test_classes],
                      'Predicted Labels': [target_labels[label] for label in rf_predictions]})

# Convert DataFrame to tabular format
table_rf = tabulate(df_rf, headers='keys', tablefmt='psql')

# Print the table
print(table_rf)

# Save the DataFrame to a CSV file
df_rf.to_csv('labels_predictions-rf-anggur.csv', index=False)

+------+--------------------------------------------------------------------------+--------------------------------------------+--------------------------------------------+
|      | Filename                                                                 | True Labels                                | Predicted Labels                           |
|------+--------------------------------------------------------------------------+--------------------------------------------+--------------------------------------------|
|    0 | 4c08f1e5-825d-4992-8cb7-0ac1bdf38ae2___FAM_L.Blight 1604_flipLR.JPG      | Grape___Leaf_blight_(Isariopsis_Leaf_Spot) | Grape___Leaf_blight_(Isariopsis_Leaf_Spot) |
|    1 | e671f7da-5f56-4409-8239-e31500250709___FAM_B.Rot 0743_flipLR.JPG         | Grape___Black_rot                          | Grape___Black_rot                          |
|    2 | b6c619c6-fd1e-41f3-80c8-8b35c2fa87ee___Mt.N.V_HL 8973.JPG                | Grape___healthy                            | G

In [55]:
# Filter rows where true label and predicted label do not match
false_predictions_rf = df_rf[df_rf['True Labels'] != df_rf['Predicted Labels']]

# Convert DataFrame to tabular format
table_rf = tabulate(false_predictions_rf, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table_rf)

# Save the false predictions to a CSV file
false_predictions_rf.to_csv('false_predictions-rf-anggur.csv', index=False)

False Predictions:
+------+---------------------------------------------------------------------+--------------------------------------------+--------------------------------------------+
|      | Filename                                                            | True Labels                                | Predicted Labels                           |
|------+---------------------------------------------------------------------+--------------------------------------------+--------------------------------------------|
|   56 | 30e108e7-3a95-401f-a815-11397d206839___FAM_B.Rot 0548.JPG           | Grape___Black_rot                          | Grape___Esca_(Black_Measles)               |
|  132 | f00f7ab3-5f23-4088-90e4-c6dcbb08333b___FAM_B.Rot 0588_flipLR.JPG    | Grape___Black_rot                          | Grape___Leaf_blight_(Isariopsis_Leaf_Spot) |
|  287 | 8c02d3db-062d-433e-b76c-b1ab7d92f6a9___FAM_B.Msls 4360.JPG          | Grape___Esca_(Black_Measles)               | Grape___Blac